In [16]:
import pandas as pd

from sudulunu.helpers import pp, dumper
import pathlib 
import os 
import json

rename = {
		"NSW": 'New South Wales',
		"QLD": 'Queensland',
		'TAS': 'Tasmania',
		'VIC':'Victoria',
		'WA':'Western Australia',
		'NT':'Northern Territory',
		'SA':'South Australia',
		"ACT": 'Australian Capital Territory'}

In [17]:
data = pd.read_csv('input/combined.csv')
# ['Date', 'Day', 'State', 'Weekday']

data['Year'] = pd.to_datetime(data['Date']).dt.strftime("%Y")

data['Count'] = 1

dicto = {}

for year in data['Year'].unique().tolist():
    inter = data.loc[data['Year'] == year].copy()
    bet = {}

    # uniques = inter.copy()
    # uniques.drop_duplicates(subset=['Date'], inplace=True)
    # pp(uniques)
    # grp = uniques.groupby(by=['Weekday'])['Count'].sum().reset_index()
    # bet['Australia'] = grp.to_json(orient='records')

    for state in inter['State'].unique().tolist():
        nother = inter.loc[inter['State'] == state].copy()
        nother.drop_duplicates(subset=['Date'], inplace=True)

        # Find holidays with "(observed)" suffix
        observed_holidays = nother[nother['Day'].str.contains(r'\(observed\)', case=False, na=False)]

        # For each observed holiday, find and remove the non-observed weekend version
        for _, obs_row in observed_holidays.iterrows():
            # Extract base holiday name (remove " (observed)")
            base_name = obs_row['Day'].replace(' (observed)', '').replace(' (Observed)', '')

            # Find the non-observed version that falls on weekend
            weekend_version = nother[
                (nother['Day'] == base_name) &
                (nother['Weekday'].isin(['Saturday', 'Sunday']))
            ]

            # Drop the weekend version
            if not weekend_version.empty:
                nother = nother.drop(weekend_version.index)

        if state == 'VIC':
            if year == "2026":
                nother.sort_values(by=['Date'], ascending=True, inplace=True)
                print(nother)
                print(nother['Count'].sum())
        grp = nother.groupby(by=['Weekday'])['Count'].sum().reset_index()
        bet[rename[state]] = grp.to_json(orient='records')
    
    dicto[year] = bet

print(dicto)
with open('calculator/static/state_counts.json', 'w') as f:
    json.dump(dicto, f, indent=4)

           Date                    Day State   Weekday  Year  Count
312  2026-01-01         New Year's Day   VIC  Thursday  2026      1
313  2026-01-26          Australia Day   VIC    Monday  2026      1
314  2026-03-09             Labour Day   VIC    Monday  2026      1
310  2026-04-03            Good Friday   VIC    Friday  2026      1
315  2026-04-04        Easter Saturday   VIC  Saturday  2026      1
316  2026-04-05          Easter Sunday   VIC    Sunday  2026      1
311  2026-04-06          Easter Monday   VIC    Monday  2026      1
317  2026-04-25              ANZAC Day   VIC  Saturday  2026      1
318  2026-06-08        King's Birthday   VIC    Monday  2026      1
319  2026-09-25        Grand Final Day   VIC    Friday  2026      1
320  2026-11-03      Melbourne Cup Day   VIC   Tuesday  2026      1
321  2026-12-25          Christmas Day   VIC    Friday  2026      1
323  2026-12-28  Boxing Day (observed)   VIC    Monday  2026      1
13
{'2025': {'Australian Capital Territory': '[{

In [18]:
### Work out consecutive days

data = pd.read_csv('input/combined.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year

years = data['Year'].unique().tolist()

# Create a full date range for all years
years_df = pd.DataFrame({
    'Date': pd.concat([pd.Series(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D'))
                        for year in years], ignore_index=True)
})
years_df['Weekday'] = years_df['Date'].dt.strftime("%A")
years_df['State'] = 'All'
years_df['Year'] = years_df['Date'].dt.year

dicto = {}

# First loop: states
for state in ['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']:
    state_name = rename[state]
    dicto[state_name] = {}
    
    first = data.loc[data['State'] == state].copy()

    # Second loop: years
    for year in years:
        second = first.loc[first['Year'] == year].copy()
        dicto[state_name][year] = {}

        # Third loop: weekdays
        for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
            dicto[state_name][year][day] = {}

            # Get weekends + this specific weekday
            inter_weekdays = ['Saturday', 'Sunday', day]
            this_years_df = years_df.loc[(years_df['Year'] == year) & (years_df['Weekday'].isin(inter_weekdays))].copy()

            # Combine holidays with weekend days
            combo = pd.concat([second, this_years_df])
            
            # Drop duplicate dates
            combo = combo.drop_duplicates(subset=['Date'])
            combo = combo.sort_values(by=['Date'], ascending=True).reset_index(drop=True)

            # Find consecutive days
            combo['DateDiff'] = (combo['Date'] - combo['Date'].shift()).dt.days
            combo['IsNewGroup'] = (combo['DateDiff'] != 1) | (combo['DateDiff'].isna())
            combo['GroupID'] = combo['IsNewGroup'].cumsum()

            # Count consecutive day lengths
            counts = {
                'Two day weekends': 0,
                'Three day weekends': 0,
                'Four day weekends': 0,
                'Five day weekends': 0
            }

            for group_id in combo['GroupID'].unique():
                group_size = len(combo[combo['GroupID'] == group_id])
                
                if group_size == 2:
                    counts['Two day weekends'] += 1
                elif group_size == 3:
                    counts['Three day weekends'] += 1
                elif group_size == 4:
                    counts['Four day weekends'] += 1
                elif group_size >= 5:
                    counts['Five day weekends'] += 1

            dicto[state_name][year][day] = counts

with open('calculator/static/consecutive_days.json', 'w') as f:
    json.dump(dicto, f, indent=4)

In [19]:
### Monday is the most common day overall

# grp = data.groupby(by=['Weekday'])['Count'].sum().reset_index()
# 0     Friday     68
# 1     Monday    169
# 2   Saturday     54
# 3     Sunday     40
# 4   Thursday     16
# 5    Tuesday     36
# 6  Wednesday     20

# pp(grp)

In [20]:
# ### Monday is the most common day next year

# grp = data.groupby(by=['Year', 'Weekday'])['Count'].sum().reset_index()

# grp = grp.loc[grp['Year'] == '2026']

# pp(grp)

In [21]:
# ### 

# grp = data.groupby(by=['Year', 'State', 'Weekday'])['Count'].sum().reset_index()

# grp = grp.loc[(grp['Year'] == '2026') & (grp['State'] == 'VIC')]


# pp(grp)

In [22]:
# vic = data.loc[data['State'] == 'VIC'].copy()
# vic = vic.loc[vic['Year'] == '2026']

# pp(vic)